# Preparation

## Import packages

In [1]:
import os, sys
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
import math
import copy
import time

In [2]:
from keras import backend as K
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers import merge, Conv2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout
from keras.layers.merge import concatenate
from keras.layers.core import Dropout, Lambda
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [3]:
# Band 1: blue; Band 2: green; Band 3: red; Band 4: nir
I = np.load(os.path.join('../data/Training.npy'))
I.shape

(4, 3344, 244, 244)

In [4]:
T = np.load('../data/Targets.npy')
T.shape

(3344, 244, 244)

In [ ]:
# Class 0 signifies nodata
classes = np.unique(T.flatten())
print(classes)

In [8]:
X = np.float32(I)
X[X>10000] = 10000
X = X / 10000.
X.shape

(4, 3344, 244, 244)

In [9]:
X = np.rollaxis(X, 0, 4)
xshape = X.shape
print(xshape)

(3344, 244, 244, 4)


In [10]:
mu = np.zeros((4,))
std = np.zeros((4,))
for q in range(X.shape[3]):
    tmp = copy.copy(X[:, :, :, q])
    tmp = tmp.flatten()
    mu[q] = tmp[tmp>0].mean()
    std[q] = tmp[tmp>0].std()
    print(mu[q], std[q])
    tmp[tmp>0] = (tmp[tmp>0] - mu[q]) / std[q]
    X[:, :, :, q] = tmp.reshape(X.shape[0], X.shape[1], X.shape[2])
    del tmp

0.12655076384544373 0.016980711370706558
0.09710454940795898 0.016959035769104958
0.07182221114635468 0.023930516093969345
0.17956602573394775 0.12075547128915787


## Check data quality

## The data quality check is to delete images that 
    - all image belongs to one class. These are often either large water bodies or distorted images
    - Images that have less then 3 classes, which are again distorted imaages

In [13]:

arr=[]
for i in range(T.shape[0]):
    if np.std(T[i,:,:])==0 or np.unique(T[i,:,:]).size<3:
        arr.append(i)
        
T=np.delete(T,arr,0)
X=np.delete(X,arr,0)
       


In [17]:
Y = np.zeros(T.shape +  (len(classes),), dtype=np.float32)
for q in range(len(classes)):
    tmp = Y[:,:,:,q]
    tmp[T==q] = 1
    Y[:,:,:,q] = tmp
print(Y.shape)

(2702, 244, 244, 9)


In [24]:
np.save('../data/X', X1)

In [25]:
np.save('../data/Y', Y1)

In [5]:
X1=np.load(os.path.join('../data/X.npy'))
Y1=np.load(os.path.join('../data/Y.npy'))

In [6]:
X1= X1[:,:128,:128,:]
Y1= Y1[:,:128,:128,:]

In [7]:
X1.shape

(2702, 128, 128, 4)

In [8]:
Y1.shape

(2702, 128, 128, 9)

# U-net

In [10]:
import keras
from keras.models import *
from keras.layers import *
from keras import layers

# Source:
# https://github.com/fchollet/deep-learning-models/blob/master/resnet50.py




def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at
                     main path
        filters: list of integers, the filterss of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters

    if IMAGE_ORDERING == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), data_format=IMAGE_ORDERING,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, data_format=IMAGE_ORDERING,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), data_format=IMAGE_ORDERING,
               name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block,
               strides=(2, 2)):
    """conv_block is the block that has a conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at
                     main path
        filters: list of integers, the filterss of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    Note that from stage 3, the first conv layer at main path is with
    strides=(2,2) and the shortcut should have strides=(2,2) as well
    """
    filters1, filters2, filters3 = filters

    if IMAGE_ORDERING == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), data_format=IMAGE_ORDERING, strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, data_format=IMAGE_ORDERING,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), data_format=IMAGE_ORDERING,
               name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), data_format=IMAGE_ORDERING,
                      strides=strides, name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(
        axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x


def get_resnet50_encoder(input_height=224,  input_width=224,
                         pretrained='imagenet',
                         include_top=True, weights='imagenet',
                         input_tensor=None, input_shape=None,
                         pooling=None,
                         classes=1000):

    assert input_height % 32 == 0
    assert input_width % 32 == 0
    inputs = keras.Input(shape=img_size )
  
    x = ZeroPadding2D((3, 3), data_format=IMAGE_ORDERING)(img_input)
    x = Conv2D(64, (7, 7), data_format=IMAGE_ORDERING,
               strides=(2, 2), name='conv1')(x)
    f1 = x

    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), data_format=IMAGE_ORDERING, strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')
    f2 = one_side_pad(x)

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')
    f3 = x

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')
    f4 = x

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')
    f5 = x

    x = AveragePooling2D(
        (7, 7), data_format=IMAGE_ORDERING, name='avg_pool')(x)
    # f6 = x

    if pretrained == 'imagenet':
        weights_path = keras.utils.get_file(
            pretrained_url.split("/")[-1], pretrained_url)
        Model(img_input, x).load_weights(weights_path)

    return img_input, [f1, f2, f3, f4, f5]



SyntaxError: invalid syntax (<ipython-input-10-6764b0b13c02>, line 30)

In [ ]:
model=get_resnet50_encoder

In [8]:
batch_size = 2
epochs = 100
callbacks = [
    keras.callbacks.ModelCheckpoint("../checkpoints/test.h5", save_best_only=True)
]
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")


In [9]:
t = time.time()

model.fit(X1, Y1, batch_size=batch_size, epochs=epochs, verbose=1, 
                     validation_split=0.2, callbacks=callbacks)
                     
print('Elapsed time for training: %.02f sec' % (time.time() - t))

Epoch 1/100
1081/1081 [==============================] - 24s 22ms/step - loss: 10.6362 - val_loss: 13.6665
Epoch 2/100
1081/1081 [==============================] - 21s 20ms/step - loss: 10.5887 - val_loss: 9.6907
Epoch 3/100
1081/1081 [==============================] - 20s 18ms/step - loss: 11.2396 - val_loss: 9.8572
Epoch 4/100
1081/1081 [==============================] - 20s 19ms/step - loss: 10.5877 - val_loss: 10.5414
Epoch 5/100
1081/1081 [==============================] - 20s 18ms/step - loss: 10.6517 - val_loss: 10.0551
Epoch 6/100
1081/1081 [==============================] - 20s 18ms/step - loss: 10.7665 - val_loss: 11.7601
Epoch 7/100
1081/1081 [==============================] - 20s 18ms/step - loss: 10.6529 - val_loss: 9.7946
Epoch 8/100
1081/1081 [==============================] - 20s 19ms/step - loss: 10.6452 - val_loss: 9.7874
Epoch 9/100
1081/1081 [==============================] - 20s 19ms/step - loss: 10.6456 - val_loss: 10.2101
Epoch 10/100
1081/1081 [=================

In [ ]:
model.save("../models/model-8Mparameters-100epoch.h5")


In [ ]:
# U-net
model.load_weights(filepath = 'model-224.h5')

score = model.evaluate(X1, Y1, verbose=0)


In [ ]:
score

# RESULTS

https://cidportal.jrc.ec.europa.eu/services/webview/jeodpp/ml-showcase/